# Installation and general imports

In [ ]:
!pip install nltk
!python -m nltk.downloader averaged_perceptron_tagger
!pip install --upgrade nltk
!pip install transformers
!pip install torch torchvision
!pip install --upgrade diffusers accelerate transformers
!pip install controlnet_aux
!pip install -q diffusers transformers accelerate invisible-watermark>=0.2.0

In [ ]:
# Import for text generation
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import gutenberg
import random
from transformers import pipeline
from enum import Enum
from transformers import BertForTokenClassification
from concurrent.futures import ThreadPoolExecutor # to avoid sequential processing

nltk.download('gutenberg')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger', download_dir='/home/jovyan/nltk_data')
nltk.data.path.append("/home/jovyan/nltk_data")

In [ ]:
# Import for image generation
import requests
from PIL import Image, ImageOps, ImageDraw
from io import BytesIO
from diffusers import StableDiffusionXLImg2ImgPipeline
from diffusers import StableDiffusionXLPipeline
from diffusers import AutoPipelineForImage2Image
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForInpainting
import torch
from diffusers.utils import load_image, make_image_grid

# Prompt generation

## Text load and analysis (adjective, noun, verb)

In [ ]:
book_complete = gutenberg.words('austen-emma.txt')
austen_emma = ' '.join([word for word in book_complete if word.isalpha()])
#pipe_bert = pipeline("token-classification", model="bert-base-uncased", device=0)
pipe_bert = pipeline("token-classification", model="vblagoje/bert-english-uncased-finetuned-pos", device=0)

In [ ]:
# Extract text from books present in the Gutemberg's library
def prompt_gutemberg(text):
    def chunk_text(text, max_length=512):
        words = text.split()  # Split text into words
        chunks = []
        current_chunk = []

        # Iterate over words and form chunks
        for word in words:
            if len(" ".join(current_chunk + [word])) <= max_length:
                current_chunk.append(word)
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [word]
                
        # Add the last chunk if it exists
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        return chunks

    # Chunk the input text
    chunks = chunk_text(text)
    
    # Initialize lists to store words by POS
    nouns = []
    verbs = []
    adjectives = []
    
    # Use ThreadPoolExecutor to process chunks concurrently
    with ThreadPoolExecutor() as executor:
        word_classifications = list(executor.map(pipe_bert, chunks))  # Process all chunks concurrently
    
    for chunk, word_classification in zip(chunks, word_classifications):
        for word in word_classification:
            pos = word.get('entity')
            word_text = word.get('word')
            
            if pos == 'NOUN':
                nouns.append(word_text)
            elif pos == 'VERB':
                verbs.append(word_text)
            elif pos == 'ADJ':
                adjectives.append(word_text)
                
    # Filter out tokenized words starting with ##
    nouns = [word for word in nouns if not word.startswith("##")]
    verbs = [word for word in verbs if not word.startswith("##")]
    adjectives = [word for word in adjectives if not word.startswith("##")]

    # Check if lists are non-empty before selecting random words
    prompt_components = []
    
    if adjectives:
        prompt_components.append(random.choice(adjectives))
    if nouns:
        prompt_components.append(random.choice(nouns))
    if verbs:
        prompt_components.append(random.choice(verbs))
    if adjectives:
        prompt_components.append(random.choice(adjectives))
    if nouns:
        prompt_components.append(random.choice(nouns))
    
    return prompt_components


## Prompt generation

In [ ]:
class PromptType(Enum):
    CONCRETE = 'concrete'
    AUSTEN = 'austen'
    WORDNET = 'wordnet'

In [ ]:
# List generated by ChatGPT of concrete, representable nouns
concrete_nouns = ["car", "dwarf", "table", "tree", "house", "dog", "cat", "mountain", "river", "bird",
"plane", "cloud", "pen", "book", "cup", "ball", "shoe", "phone", "clock", "camera","lamp", "hat", "bicycle", "rocket", "laptop", "bottle", "fish", "star", "candle", "watch",
"guitar", "keyboard", "umbrella", "bridge", "window", "door", "broom", "jacket", "television",
"bookcase", "television", "computer", "flute", "telescope", "skateboard", "drum", "ice cream",
"piano", "pizza", "painting", "carpet", "treehouse", "swimming pool", "skyscraper", "mountain",
"statue", "wall", "stool", "frame", "chandelier", "refrigerator", "oven", "microwave", "furniture",
"ladder", "stone", "trolley", "balloon", "pillow", "backpack", "scooter", "nail", "lightbulb",
"zebra", "tiger", "elephant", "rabbit", "pencil", "cupboard", "brush", "chair", "scissors", "cone",
"brush", "van", "helicopter", "carousel", "whistle", "marble", "plank", "skateboard", "rollerblades",
"cheese", "rose", "lighthouse", "plane", "whale", "doll", "boat", "toaster", "telephone", "skateboard",
"daisy", "tree stump", "sword", "axe", "glove", "mushroom", "pinecone", "turtle", "snake", "donkey",
"bat", "truck", "lizard", "pot", "teapot", "goblet", "beer", "straw", "shovel", "scooter", "bicycle",
"shark", "whale", "lemon", "coffee cup", "cactus", "mug", "helmet", "screwdriver", "camera", "snowflake",
"computer", "windmill", "vase", "lantern", "bathtub", "piano", "mug", "wheelbarrow", "giraffe", "monkey",
"leaf", "spoon", "carrot", "cake", "trophy", "book", "card", "glasses", "socks", "shoe", "box",
"note", "purse", "blanket", "airplane", "beach", "forest", "bike", "plate", "sand", "barbecue",
"bathtub", "shovel", "lawnmower", "trowel", "seashell", "volcano", "tulip", "coffin", "couch", "bread",
"jar", "flower", "acorn", "crab", "surfboard", "whip", "lighthouse", "earphones", "towel", "knives",
"comb", "gamepad", "scissors", "drawer", "cork", "socks", "sunflower", "sunhat", "glove", "bookstand",
"glasses", "mountainbike", "glove", "frying pan", "plate", "marbles", "guitar", "wrench", "t-shirt",
"cat", "dog", "hedgehog", "train", "bridge", "cannon", "kettle", "skateboard", "basketball", "sculpture",
"nut", "spade", "teaspoon", "pen", "camera", "sleeping bag", "fence", "frying pan", "tennis ball", "cube",
"alarm clock", "guitar", "wheel", "hammer", "skate", "drone", "broom", "zebra", "rubberband", "television",
"giraffe", "bear", "raccoon", "purse", "thimble", "fork", "toothbrush", "paper", "spoon", "trophy",
"tent", "candle", "guitar", "flower", "robot", "monitor", "speaker", "couch", "floor", "bus", "ball",
"drumstick", "banana", "fork", "tissue", "coat", "hammock", "mug", "calculator", "hoop", "garden", "sailboat",
"flute", "pin", "lightbulb", "notepad", "seashell", "keyboard", "plow", "airplane", "wheelchair", "iron",
"baseball", "lock", "glove", "pencil", "lawnmower", "book", "ticket", "trolley", "hat", "leaf", "fish",
"ring", "rock", "gloves", "umbrella", "drum", "wrench", "football", "knife", "cloud", "pineapple", "ladle",
"pear", "truck", "shell", "van", "bicycle", "doorbell", "wood", "bat", "cleaver", "whale", "sandwich",
"scarf", "mattress", "bucket", "dishes", "drawer", "clock", "mailbox", "planet", "pizza", "doghouse",
"marshmallow", "jacket", "parachute", "seesaw", "balloon", "teddy bear", "sand", "glue", "mat",
"capsule", "air", "rainbow", "toy", "chicken", "wrench", "tablecloth", "tractor", "flute", "fence",
"saw", "snowman", "pocket", "stairs", "flag", "flowerpot", "ruler", "brick", "hula hoop", "castle",
"hamster", "pear", "lighthouse", "skyscraper", "tree", "ring", "candy", "fridge", "lemon", "fishbowl",
"cloud", "bed", "trolley", "backpack", "dish", "cup", "boot", "rope", "bat", "telescope", "barrel",
"pillow", "cardboard", "snowball", "cherry", "rocket", "toothpaste", "puzzle", "bat", "flag", "stone",
"toys", "giraffe", "skateboard", "cactus", "hammer", "cupboard", "mountain", "lamp", "fish", "keyboard",
"snowflake", "brush", "mug", "roller", "ice", "pear", "snow", "fence", "moon", "marker", "glue",
"basket", "frisbee", "paintbrush", "guitar", "glass", "french fries", "key", "seesaw", "bathtub",
"dinosaur", "truck", "umbrella", "ferris wheel", "puppy", "sweater", "hat", "windmill", "needle",
"train", "windmill", "brush", "apple", "egg", "star", "stapler", "smoke", "sand", "planet", "drum",
"broom", "brick", "ladle", "bat", "wooden spoon", "scissors", "umbrella", "camera", "dish", "brush", "airship", "apple", "axe", "balloon", "bathtub", "bell", "bicycle", "birdcage", "bison", "blender",
"bookend", "boot", "boxer", "bucket", "cactus", "calculator", "camera", "candleholder", "cannon", "caterpillar",
"chalk", "chandelier", "cheese", "chicken", "chimney", "chisel", "christmas tree", "circular saw", "clamp", "clock",
"clothesline", "coconut", "cone", "cookie", "corn", "couch", "cucumber", "curtain", "dagger", "desk",
"dinosaur", "dishwasher", "door", "doorway", "doughnut", "drill", "drumstick", "earphones", "eggplant", "electric fan",
"elephant", "envelope", "eraser", "eye", "feather", "fence", "ferris wheel", "fishing rod", "flame", "flashlight",
"floor lamp", "flower", "flowerpot", "fork", "frying pan", "funnel", "giraffe", "glove", "glue gun", "golf ball",
"grape", "guitar", "hair dryer", "hamburger", "hammer", "hammock", "hat", "headphones", "helicopter", "hose",
"houseplant", "ice cream cone", "jackal", "jacket", "jigsaw", "joystick", "jump rope", "kettle", "keyboard", "knife",
"koala", "ladle", "lawn mower", "leaf blower", "lemon", "lightbulb", "lighthouse", "lizard", "lock", "lollipop",
"luggage", "magnifying glass", "map", "mattress", "measuring cup", "megaphone", "melon", "mirror", "mop", "mug",
"nail", "necklace", "notebook", "nugget", "paddle", "paintbrush", "palette", "palm tree", "paperclip", "parachute",
"pencil case", "pet house", "phone charger", "piano", "ping pong paddle", "pirate ship", "pistol", "plankton", "plate",
"plow", "plug", "pocket watch", "police car", "pothole", "puzzle", "radio", "rake", "remote control", "ring",
"rocket", "roller coaster", "ruler", "sandwich", "saw", "scissors", "seashell", "shovel", "skateboard", "skillet",
"skis", "sled", "slingshot", "snowboard", "soap", "soccer ball", "spatula", "sponge", "spoon", "staircase",
"stethoscope", "stopwatch", "sunglasses", "sushi", "swim goggles", "sword", "syringe", "telescope", "television",
"tennis racket", "thermometer", "thimble", "tie", "toaster", "toothbrush", "toothpaste", "towel", "traffic light",
"trampoline", "trolley", "trophy", "truck", "tulip", "vacuum cleaner", "van", "vase", "velvet rope", "video camera",
"violin", "volcano", "wrench", "xylophone", "yarn", "yoga mat", "zebra", "zipper", "zucchini", "zombie",
"accordion", "air conditioner", "alarm clock", "ambulance", "anvil", "aquarium", "art easel", "ashtray", "avocado",
"axe handle", "backpack", "banana", "bark", "barn", "barometer", "barrel", "basket", "bat", "beacon", "beer bottle",
"bicycle helmet", "binoculars", "bird nest", "blackboard", "blender", "block", "bowl", "box", "bracelet", "bricks",
"broccoli", "broom", "bucket hat", "bunker", "bus", "butterfly", "cabinet", "calculator", "calendar", "camera bag",
"candle", "cap", "car tire", "carpet", "carton", "case", "casket", "castle", "catapult", "cattle", "caveman", "cell phone",
"cello", "chainsaw", "chalkboard", "cheese grater", "chessboard", "chicken coop", "child", "cigar", "circuit board", "clamp",
"clay pot", "clip", "clothes", "coffee maker", "coffin", "coin", "colored pencils", "compass", "cone hat", "cookie jar",
"cork", "couch cushion", "cupboard", "curtains", "dart", "desk lamp", "dinosaur skeleton", "dirt bike", "dog bowl",
"dog collar", "door knob", "doorknob", "doorframe", "dough", "drain", "drawer", "drum set", "dumbbell", "duster",
"dustpan", "earrings", "elbow", "envelope opener", "eraser", "exercise ball", "fence post", "ferry", "file folder",
"fire hydrant", "fishing tackle", "flamingo", "flute", "fog", "frying pan", "garage", "gasoline", "golf cart",
"grill", "guitar pick", "hairbrush", "hats", "headset", "holster", "hook", "hopscotch", "hula skirt", "ice skates",
"ice tray", "incense", "island", "jar lid", "jewels", "jockey cap", "jungle gym", "keyboard keys", "kiwi", "ladle",
"leaf blower", "lemonade", "light switch", "lollipop", "luggage rack", "magic wand", "magnet", "mailbox", "mannequin",
"maple tree", "market", "mash potatoes", "mason jar", "measuring tape", "melons", "menorah", "mirror frame", "mobile",
"monopoly board", "moped", "mousepad", "muffin", "nachos", "nail polish", "neck pillow", "nightstand", "nutcracker",
"office chair", "paint roller", "paper bag", "paper towel", "paperweight", "parrot", "peach", "pedal", "penholder",
"phone case", "piano keys", "picnic basket", "pie", "picture frame", "pin", "potted plant", "printer", "purse",
"rake", "refrigerator", "remote", "rubber duck", "sand bucket", "sandpaper", "scarf", "screwdriver", "shampoo",
"shopping cart", "shovel", "sidewalk", "skate", "scooter", "scrub brush", "seashell", "selfie stick", "shoe rack",
"shopping bag", "skull", "snow globe", "snowman", "sock", "spade", "sponge", "spoon", "sports car", "spring",
"staircase", "steak", "stereo", "stethoscope", "strawberry", "street sign", "stripes", "submarine", "sunglasses",
"surfer", "swan", "swimming goggles", "sword", "t-shirt", "tablecloth", "teddy bear", "telephone", "telescope",
"tent", "thermometer", "ticket", "tiger", "toaster", "toothpaste", "toothpick", "traffic cone", "train tracks",
"tree stump", "tricycle", "trophy", "trolley", "tulip", "umbrella", "vacuum", "van", "video game controller", "violin",
"vulture", "washer", "water bottle", "webcam", "whale", "wheel", "window", "window blind", "wooden crate", "woodpecker",
"yarn", "yoga ball", "zeppelin", "zipper", "zucchini", "accordion", "air balloon", "airplane wing", "airship", "alarm", "aloe vera", "ambulance", "amplifier", "anatomy model",
"anchor", "angel", "ant", "anvil", "apron", "arcade", "armor", "art canvas", "artichoke", "ashtray", 
"asteroid", "atlas", "automatic door", "autumn leaf", "awning", "baby bottle", "baby stroller", "backhoe", "baker's rack", 
"balance scale", "banana peel", "bandage", "banknote", "barbecue grill", "barbell", "barn door", "barrow", "baseball glove", 
"baseball bat", "basketball hoop", "bathtub faucet", "beach ball", "beach chair", "beaker", "beanie", "beer mug", "bicycle tire", 
"billiard ball", "binder", "bin", "birdhouse", "biscuit", "bison skull", "blackboard eraser", "blanket", "blender bottle", 
"blindfold", "block of ice", "blood pressure cuff", "blowtorch", "blueberry", "boiling pot", "bottle opener", "bowtie", 
"bracelet box", "brandy glass", "bread loaf", "broomstick", "bruschetta", "bucket list", "buffalo", "bunk bed", "bus stop",
"cabana", "cactus plant", "calculator keyboard", "calendar page", "candle jar", "candy cane", "carabiner", "card catalog", 
"carpet roll", "carousel horse", "carving knife", "cassette tape", "casserole dish", "cat tree", "cattle prod", "cellar", 
"cereal box", "chalice", "chalkboard eraser", "chandelier light", "cheese wedge", "cheeseburger", "chess piece", "chicken leg", 
"chicken nugget", "chimney sweep", "chipmunk", "chisel set", "christmas stocking", "cigar box", "cinnamon stick", "claw hammer", 
"clay pot", "clenched fist", "clothes hanger", "clothes iron", "clothesline peg", "cloud formation", "clown wig", "coaster set", 
"coffin lid", "coin purse", "coffee filter", "coffee table", "coin collection", "compass rose", "conga drum", "cooking pot", 
"corkscrew", "cotton ball", "countertop", "cowboy hat", "crayon box", "cricket bat", "cromlech", "crown", "cup holder", "cup of soup", 
"cushion", "cutting board", "dartboard", "deck chair", "decorative plate", "desk organizer", "detergent bottle", "diamond ring", 
"dinosaur tail", "dining table", "dipper", "dish rack", "doughnut box", "drainpipe", "dressing gown", "drinking glass", 
"drumsticks", "drunken sailor", "duck", "dustpan", "electric drill", "elephant trunk", "envelope seal", "eraser top", "exam paper", 
"exercise mat", "ferris wheel seat", "field hockey stick", "filing cabinet", "fingerless glove", "fishing lure", "fishing pole", 
"flask", "flashlight bulb", "flip flops", "flower basket", "flower bed", "flute case", "football helmet", "football shoes", 
"footlocker", "french press", "frying pan handle", "fudge", "funnel spout", "garden gnome", "gas mask", "gavel", "generator", 
"george washington statue", "giraffe print", "glass bottle", "glove compartment", "goggles", "gold medal", "golf tee", "grape vine", 
"grill brush", "guitar neck", "guitar strap", "hammered copper", "hammock frame", "hand fan", "hand mixer", "hand towel", 
"harp", "hat rack", "hazmat suit", "headband", "heart shaped balloon", "helmet liner", "herb garden", "hiking boots", 
"hip flask", "hoop skirt", "hummingbird feeder", "hurricane lamp", "ice cream bar", "ice pack", "ice tray", "incense holder", 
"information board", "ink bottle", "iron gate", "iron skillet", "jack-o'-lantern", "jacket zipper", "jewelry box", "jigsaw puzzle", 
"kettle lid", "keyhole", "keypad", "keyboard keys", "kiwi fruit", "ladle spoon", "lantern handle", "lawn chair", "leather belt", 
"lemon zest", "library card", "lifeboat", "light bulb socket", "lighter", "lightning bolt", "lily pad", "locket", "log cabin", 
"luggage tag", "lumberjack shirt", "lunch box", "magician's hat", "magnifying lens", "mail envelope", "mail slot", "map pin", 
"mattress pad", "mechanical pencil", "medal holder", "megaphone horn", "measuring cup set", "measuring jug", "milk jug", 
"mirror frame", "miter box", "model airplane", "monitor stand", "mop bucket", "mosquito net", "motion sensor", "muffin tray", 
"neck pillow", "netting", "night lamp", "oil can", "oil paint", "olives", "onion ring", "open sign", "orange peel", "ornament", 
"outdoor bench", "pack of cards", "painting easel", "paper fan", "paper lantern", "parasol", "pen holder", "pepper shaker", 
"picnic table", "pillowcase", "pine cone", "pinwheel", "plate holder", "playground slide", "plunger", "pocket mirror", 
"popcorn bucket", "postcard", "power cord", "puzzle piece", "quilt", "radiator", "raincoat", "rake handle", "rainy day boots", 
"recipe book", "reclining chair", "remote control", "ring binder", "roller skate", "rolling pin", "rooftop", "rug", "salad bowl", 
"salt shaker", "sandpaper block", "sandwich board", "scarf hanger", "scissors handle", "scratching post", "shack", "shaker", 
"shampoo bottle", "shaving kit", "shoelace", "shopping bag", "shopping trolley", "shovel head", "side table", "sieve", 
"skateboard deck", "skewers", "sled runners", "slippers", "slingshot band", "snowboard goggles", "snowflake ornament", 
"soap dispenser", "soda can", "spade handle", "spatula handle", "speech bubble", "speedometer", "spoon holder", "sports car",
"spring blossom", "stainless steel mug", "stapler", "straw", "sugar bowl", "sunglass case", "sushi roll", "swimming float", 
"sword handle", "tablecloth corner", "tape measure", "teacup", "teapot lid", "telephone receiver", "telescope lens", "thermos", 
"ticket stub", "toaster slot", "toothbrush holder", "toothpick container", "traffic cone", "trailer hitch", "trampoline net", 
"trash can lid", "treasure chest", "trolley wheel", "trophy stand", "truck bed", "turkey baster", "typewriter keys", 
"vacuum cleaner hose", "vase flower", "vinyl record", "vintage camera", "volleyball net", "watering can", "webcam lens", 
"whistle", "windsurfing board", "window box", "window shutter", "wooden crate", "yarn spool", "yellow marker", "zip bag", "zodiac sign", "abacus", "accordion", "air balloon", "air conditioner", "alarm clock", "alligator", "ambulance", "anatomy model", "anchor", 
"angel", "antique clock", "apron", "art easel", "atlas", "autumn leaf", "aviator sunglasses", "backpack", "bacon strip", 
"baking tray", "ball of yarn", "banana split", "bandana", "basket of apples", "bat", "bathing suit", "bathtub", "beach ball", 
"beacon", "beach umbrella", "beanie", "beer can", "bench", "bicycle wheel", "bird bath", "birdhouse", "biscuit tin", "blender", 
"blinds", "block of wood", "block tower", "blueberry pie", "bottle cap", "bottle of ketchup", "bottle of wine", "bowl of soup", 
"brass lamp", "bread basket", "broom", "broccoli bunch", "buckets", "bunk bed", "cactus plant", "calculator", "calendar", "camera", 
"can opener", "candy bar", "candle stick", "candle holder", "car battery", "car wheel", "carabiner", "card deck", "casserole dish", 
"cat tree", "cattle", "chandelier", "chess board", "chicken wing", "chip bag", "chisel", "christmas wreath", "cigarette", 
"clamp", "claw", "clothes iron", "clothes line", "clothing hanger", "clutch", "cobweb", "coconut", "coffee grinder", "coffee mug", 
"coffee pot", "coin stack", "coffee table", "comb", "computer mouse", "computer screen", "concrete block", "cone", "cookie cutter", 
"cookie jar", "cork", "corkscrew", "cotton candy", "crayon box", "crossword puzzle", "cucumber", "cufflinks", "cupcake", 
"curtain rod", "cushion", "dartboard", "darts", "deck chair", "decorative pillow", "dog bowl", "dog collar", "dog house", 
"dollhouse", "doughnut", "drainpipe", "drawer handle", "drumstick", "dumbbell", "dustpan", "earphones", "earrings", 
"easel", "egg carton", "electric fan", "electric guitar", "elephant", "envelope", "eraser", "exercise mat", "extension cord", 
"feather duster", "fence post", "file folder", "fishing rod", "flask", "flashlight", "flip-flops", "flower bouquet", 
"flower pot", "flying saucer", "frying pan", "garden rake", "garden shovel", "gasoline can", "globe", "glue gun", "goggles", 
"golf ball", "golf club", "golf tee", "grape vine", "gravy boat", "grill", "guitar pick", "hammock", "hand fan", "hat", 
"headphones", "helicopter", "hiking boots", "hip flask", "hole puncher", "hula hoop", "ice cream cone", "ice tray", 
"incense holder", "inhaler", "ink pad", "iron", "jack-o-lantern", "jacket", "jelly jar", "jigsaw puzzle", "jockey cap", 
"jumper cable", "keyboard", "kiwi", "ladle", "landline phone", "laptop", "lawnmower", "leaf blower", "letter opener", 
"lemon", "lemon squeezer", "light bulb", "lighter", "light switch", "lighthouse", "lipstick", "lollipop", "luggage", 
"magnifying glass", "mailbox", "mantle clock", "measuring cup", "measuring spoon", "melon", "microphone", "mirror", 
"mop", "mouse pad", "muffin tray", "nail polish", "nightstand", "notebook", "oar", "oil can", "olive jar", "paint bucket", 
"paintbrush", "palette", "paper clips", "paper towel", "paperweight", "parasol", "passport", "pencil", "pencil case", 
"pendant", "perfume bottle", "phone case", "photo frame", "piano", "ping pong paddle", "plate", "platter", "plunger", 
"pocket knife", "pocket watch", "pot", "power drill", "printer", "propeller", "purse", "radio", "raincoat", "recliner", 
"remote control", "reusable water bottle", "riding boots", "ring", "roller skates", "rolling pin", "rose", "ruler", 
"salt shaker", "sandwich", "scarf", "scissors", "sewing kit", "shampoo bottle", "shaving razor", "shopping bag", 
"shopping cart", "shovel", "skateboard", "scooter", "scroll", "screwdriver", "seashell", "selfie stick", "shovel", 
"skateboard deck", "sleeping bag", "snow shovel", "snowboard", "sock", "socks", "spade", "spatula", "speakers", 
"spice rack", "spoon", "sports car", "spring", "stadium", "staircase", "stapler", "stethoscope", "stepladder", 
"straw", "suitcase", "sunglasses", "surfboard", "sushi roll", "swimming pool", "swimming float", "table", "tablecloth", 
"tape measure", "tea kettle", "teacup", "teddy bear", "telephone", "television", "tennis racket", "thermometer", 
"thumbtacks", "ticket", "toaster", "toothbrush", "toothpaste", "toothpick", "traffic cone", "trolley", "trumpet", 
"umbrella", "vacuum cleaner", "van", "vase", "vegetable peeler", "violin", "volleyball", "wrench", "wristwatch", 
"xylophone", "yarn", "yellow umbrella", "yoga mat", "yo-yo", "zebra", "zipper", "ziplock bag", "zucchini", 
"air compressor", "apple pie", "atlas", "baking sheet", "bandages", "basketball", "beach chair", "bird feeder", 
"birthday hat", "bluebird", "bookmark", "bracelet", "bricks", "broomstick", "candle", "car charger", "casserole", 
"ceramic mug", "chess set", "chimney", "circle", "clothes basket", "clown nose", "clothes rack", "coffee machine", 
"coffin", "corkscrew", "cowboy boots", "crepe", "cup of coffee", "cup of tea", "curtain", "dandelion", "dog collar", 
"donut", "door lock", "door frame", "drawing compass", "earmuffs", "egg beater", "electric toothbrush", "envelope", 
"exercise bike", "fire extinguisher", "flamingo", "flute", "funnel", "game controller", "gasoline tank", "golf club", 
"guitar pick", "hairbrush", "hammock", "handbag", "hatchet", "hippopotamus", "hot dog", "hot water bottle", "iPad", 
"kettle", "kitchen sink", "lawn mower", "lemonade pitcher", "light fixture", "magnet", "mask", "microwave", 
"muffin tin", "music box", "painter's palette", "paper bag", "paper towel", "parachute", "pen holder", "phone stand", 
"photo album", "pine tree", "plate rack", "pot of tea", "printer ink", "puzzle", "quilt", "rake", "rocket", 
"roller coaster", "rubber stamp", "scissors handle", "skillet", "snow globe", "soap dish", "soccer ball", "sock", 
"spade", "spoon", "sports car", "spring", "staircase", "stapler", "stethoscope", "stepladder", "straw", "suitcase", 
"sunglasses", "surfboard", "sushi roll", "swimming pool", "swimming float", "table", "tablecloth", "tape measure", 
"tea kettle", "teacup", "teddy bear", "telephone", "television", "tennis racket", "thermometer", "thumbtacks", 
"ticket", "toaster", "toothbrush", "toothpaste", "toothpick", "traffic cone", "trolley", "trumpet", "umbrella", 
"vacuum cleaner", "van", "vase", "vegetable peeler", "violin", "volleyball", "wrench", "wristwatch", "xylophone", 
"yarn", "yellow umbrella", "yoga mat", "yo-yo", "zebra", "zipper", "ziplock bag", "zucchini"]

wordnet_nouns = list(wn.all_synsets('n'))
austen_emma = ' '.join(word for word in gutenberg.words('austen-emma.txt') if word.isalpha())

In [ ]:
# PROMPT GENERATION
def generate_prompt(prompt_type:PromptType, length):
    """ Returns a list with the words for the prompt."""
    prompt = []

    if prompt_type == PromptType.CONCRETE:
        prompt = [random.choice(concrete_nouns) for x in range(length)]

    elif prompt_type == PromptType.WORDNET:
        prompt = [random.choice(wordnet_nouns).lemmas()[0].name() for x in range(length)]

    elif prompt_type == PromptType.AUSTEN:
        # Uses above function to generate adj, noun, verb, adj, noun prompt from Book "Emma".
        prompt = [word for x in range(length) for word in prompt_gutemberg(austen_emma)]

    return prompt

## Prompt testing

In [ ]:
# UNCOMMENT TO GENERATE PROMPTS
random.seed()
"""prompt0 = generate_prompt(PromptType.CONCRETE, 2)
prompt1 = generate_prompt(PromptType.WORDNET, 2)
prompt2 = generate_prompt(PromptType.AUSTEN, 2)
print(f'Concrete prompt: {prompt0}')
print(f'Wordnet prompt: {prompt1}')
print(f'Austen prompt: {prompt2}')"""

# Image generation

Load model

In [ ]:
txt_2_img_pipe = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

In [ ]:
from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

# from_pipe to avoid consuming additional memory when loading a checkpoint
img2img_pipe = AutoPipelineForImage2Image.from_pipe(txt_2_img_pipe).to("cuda")

In [ ]:
# Prompt preparation
positive_prompt = 'clean lines, well-balanced composition, graphic'
negative_prompt = ('blurry, text, letters, letter, typography, writing, low resolution, watermark, '
                                'patterns, distorted face, bad anatomy, uncanny, extra fingers, '
                                'missing fingers, missing faces, pixelated, deformed, no text')

Text-to-image model. This model is used as a first step for all the exquisite corpse generations.

In [ ]:
def first_img(word):
    """Generates an image based on a word.
    The image is saved with the word in its name.
    Returns the filename."""
    image_1 = txt_2_img_pipe(word + '++' + positive_prompt, 
                             negative_prompt=negative_prompt, 
                             guidance_scale=13,
                             num_inference_steps=80, 
                             height=1024, 
                             width=1024).images[0]
    image_name = f'image_{word}.jpg'
    image_1.save(image_name)
    
    # Show the image
    print(word)
    print('First image generated.')
    return image_name

In [ ]:
# UNCOMMENT TO TEST first_img() function
"""random.seed(42)
prompt0 = generate_prompt(PromptType.CONCRETE, 2)
first_img(prompt0[0])"""

In [ ]:
from PIL import Image, ImageDraw, ImageFont
# This function is used in the gif pipeline

def add_text_to_image(image_path, text):
    """Add black text on top left of the image. Saves and overwrites the input image.
    Input: image_path, text.
    Returns: none."""
    
    # Open the image
    img = Image.open(image_path)
    
    # Create a drawing context
    draw = ImageDraw.Draw(img)
    
    try:
        font = ImageFont.truetype("font/Roboto-Medium.ttf", size=25) # Font in repository folder
    except IOError:
        font = ImageFont.load_default()
        
    text_position = (15, 15)  # (0,0) is the top left corner
    text_color = (0, 0, 0)  # Black text

    # Add text to the image
    draw.text(text_position, text, font=font, fill=text_color)

    # Save the modified image
    img.save(image_path)

    return "Text added"

In [ ]:
# Test - UNCOMMENT AND REPLACE WITH YOUR IMAGE PATH
"""your_image = 'image_block tower.jpg'
add_text_to_image(your_image, 'Write your text here')"""

## Gif functions

Gif creator
Source: https://medium.com/@theriyasharma24/creating-gifs-from-images-using-python-88946aa47881

In [ ]:
def gif_img_generator(img_path, words):
    """Input: previous image and word. Iterates on all prompt's images.
    Saves the images.
    Return: filenames of the generated images."""
    
    # Set the initial image as the starting point
    current_image = Image.open(img_path).convert("RGB")

    filenames = [img_path]
    outputs = [img_path]
    # Iterate over each word in the prompt
    for i in range(1, len(words)):
    
    # Uncomment for sliding window:
    #for i in range(1, (len(words)-3)):
        
        output = img2img_pipe(
            # Uncomment for sliding window:
            #prompt= (words[i]+ words[i+1] + words[i+2])+ '++' + positive_prompt,
            prompt= words[i]+ '++' + positive_prompt,
            negative_prompt=negative_prompt,
            image=current_image, 
            strength=0.8, # How much the original image is preserved
            guidance_scale=9 # How closely the image should match the prompt
        ).images[0]
    
        # Save the current output image
        output_filename = f"gif_{i+1}_{words[i]}.jpg"
        # Sliding window for Austen types
        #output_filename = f"gif_{i+1}_{words[i]}-{words[i+1]}-{words[i+2]}.jpg"
        filenames.append(output_filename)
        print('filename appended')
        output.save(output_filename)
        outputs.append(output)
        # Use the current output image as the input for the next iteration
        current_image = output
        print(f"Saved image: {output_filename}")

    print("Gif image generation done.")
    return filenames


def create_gif(image_paths, output_gif_path, duration):
    """Generates a gif with the given images."""

    images = [Image.open(image_path) for image_path in image_paths]
    
    # Save as GIF
    images[0].save(
        output_gif_path,
        save_all=True,
        append_images=images[1:],
        duration=duration,
        loop=0 # 0 means infinite loop
    )
    return ".gif file generated"

In [ ]:
# UNCOMMENT TO TEST THE GIF FUNCTIONS
#filenames_gif = gif_img_generator("image_block tower.jpg", prompt0[1:])
#gif = create_gif(filenames_gif, 'gif_test0.gif', 100*len(filenames_gif))

In [ ]:
def gif_pipeline(prompt, with_text=False):
    """ Generates an image per word of the prompt and assembles images into a gif. """
    first_img_name = first_img(prompt[0])
    filenames_gif = gif_img_generator(first_img_name, prompt[1:]) 
    if with_text:
        add_text_to_image(first_img_name, prompt[0])
        i = 1
        for filename in filenames_gif:
            add_text_to_image(filename, prompt[i])
            i+=1
    gif_name = ''.join([word.capitalize() for word in prompt])
    print(filenames_gif)        
    gif = create_gif(filenames_gif,f'{gif_name}.gif', 100*len(filenames_gif))
    return 'Gif pipeline end.'


## Gif generation example

In [ ]:
#random.seed(13)
concrete_prompts = []
for x in range(3):
    prompt = generate_prompt(PromptType.CONCRETE, 20)
    concrete_prompts.append(prompt)

In [ ]:
print(concrete_prompts[0])

In [ ]:
for prompt in concrete_prompts:
    gif_pipeline(prompt, 0)

In [ ]:
# To try sliding window
#austen_prompts = generate_prompt(PromptType.AUSTEN, 3) # Sliding window = 3 
#print(f'Check if sliding window pipeline will work: {len(austen_prompts) / 5 == 0}' )
#gif_pipeline(austen_prompts)

## Expanded images

In [ ]:
def expand_and_merge(input_img, words):
    """Crops the input image, extends it into 1024x1024, uses it as input for the next image generation. 
    Merges all images into one scrollable image."""
    input_img = Image.open(input_img).convert("RGB")
    expanded_images = [input_img]
    expanded_size = (input_img.width, input_img.height)
    #top_part = input_img.crop((0, 0, input_img.width, input_img.height // 4))
    #top_part = top_part.resize(expanded_size)
    bottom_part = input_img.crop((0, input_img.height - input_img.height // 4, input_img.width, input_img.height))
    bottom_part = bottom_part.resize(expanded_size)
    
    for word in words: #generate an image per word
        print(word)
        expanded_image = img2img_pipe(
            prompt=(word)+'++' + positive_prompt,
            negative_prompt=negative_prompt,
            image=bottom_part,
            strength=0.75,
            guidance_scale=9).images[0] # 15 7.5
        
        expanded_images.append(expanded_image)
        bottom_part = expanded_image.crop((0, input_img.height - input_img.height // 4, input_img.width, input_img.height))
        bottom_part = bottom_part.resize((1024, 1024))

    total_height = input_img.height * (len(expanded_images))
    width = input_img.width  # All images have the same width

    # Create a new canvas to hold all images stacked vertically
    merged_image = Image.new("RGB", (width, total_height))

    # Paste each image into the canvas at the correct position
    current_height = 0
    for image in expanded_images:
        merged_image.paste(image, (0, current_height))
        current_height += image.height
    len_prompt = [len(word) for word in prompt]
    total_len_prompt = sum(len_prompt)
    if total_len_prompt < 60: # Prevent crashes due to long filenames
        merged_png = merged_image.save(f"merged_{prompt}.png")
    else:
        merged_png = merged_image.save(f"merged_{prompt[:3]}_&more.png")
    print('Expand and merge done.')
    return merged_image

In [ ]:
def extend_pipeline(prompt):
    """Generates one long image 1024x(len(prompt))"""
    first_img_name = first_img(prompt[0])
    merged_img = expand_and_merge(first_img_name, prompt[1:])
    print('Extend pipeline done.')
    return merged_img


## Expand and merge test

In [ ]:
random.seed(42)
prompts_concrete = []
for i in range(3):
    prompt_concrete = generate_prompt(PromptType.CONCRETE, 10)
    prompts_concrete.append(prompt_concrete)

In [ ]:
for prompt in concrete_prompts:
    extend_pipeline(prompt)

## Inpainting

In [ ]:
inpainting_pipe = AutoPipelineForInpainting.from_pipe(txt_2_img_pipe).to("cuda")

In [ ]:
def inpainting(source_img_path, word, mini_size:int, mask_size:int):
    """Pastes a small copy of the source image in its center and extends it. The centered image remains unchanged.
    Returns the generated image's name."""
    canvas_width, canvas_height = 1024, 1024
    source_image = Image.open(source_img_path).convert("RGB")
    source_image_mini = source_image.resize((mini_size, mini_size))
    x_offset = (canvas_width - source_image_mini.width) // 2
    y_offset = (canvas_height - source_image_mini.height) // 2
    source_image.paste(source_image_mini, (x_offset, y_offset))

    
    mask = Image.new("L", (canvas_width, canvas_height), 255) # L for luminance --> grayscale mode                 
    mask.paste(0, (x_offset, y_offset, x_offset + mask_size, y_offset + mask_size))
    print('mask generated')
    
    img_inpainting = inpainting_pipe(prompt=word+ "++" + positive_prompt,
                                     negative_prompt=negative_prompt,
                                     image=source_image, 
                                     mask_image=mask,
                                     guidance_scale=6
                                    ).images[0]

    filename = f'inpainting_{source_img_path[11:-4]}_{word}.png'
    img_inpainting.save(filename)

    print(f'Inpainting done: {filename} saved ')
    return filename

In [ ]:
def inpainting_pipeline(prompt):
    """Input: full prompt (type: list)
       Ouput: filenames
       This function generates an exquisite corpse with previous images in the middle of the final output. 
       All images are saved separately.""" 
    first_img_name = first_img(prompt[0])
    zoom_factor = 1024//len(prompt)
    mini_size = zoom_factor 
    mask_size = zoom_factor - 50
    filenames = []
    for word in prompt[1:]:
        inpainting_img = inpainting(first_img_name, word, mini_size, mask_size)
        filenames.append(inpainting_img)
        first_img_name = inpainting_img
        print(f'inpainting \'{word}\' done')
        mini_size += zoom_factor
        mask_size += zoom_factor
        
    print('Inpainting pipeline done')
    return filenames
    

In [ ]:
# UNCOMMENT TO TEST
"""random.seed(2)
prompts_concrete = []
for i in range(3):
    prompt_concrete = generate_prompt(PromptType.CONCRETE, 6)
    prompts_concrete.append(prompt_concrete)

for prompt in prompts_concrete:
    inpainting_pipeline(prompt)"""

## Video

Code adapted from https://github.com/bermarte/Zoom-in-video

In [ ]:
!pip install moviepy==1.0.3 numpy>=1.18.1 imageio>=2.5.0 decorator>=4.3.0 tqdm>=4.0.0 Pillow>=7.0.0 scipy>=1.3.0 pydub>=0.23.0 audiofile>=0.0.0 opencv-python>=4.5

In [ ]:
import os
from moviepy.editor import ImageClip, concatenate_videoclips
from moviepy.editor import *

In [ ]:
import os
from moviepy.editor import ImageClip, concatenate_videoclips

def create_zoom_video(images, output_file, zoom_factor=0.05, duration_per_image=1):
    '''It takes a list of input image files, zooms them in progressively to create a clip'''
    clips = []
    print('clip list done')
    # Calculatfinal_clip = final_clip.set_fps(25)e the zoom values
    for _i, image in enumerate(images):
        print('for loop entered')
        # Load the image as a VideoClip
        clip = ImageClip(image)
        print('image loaded')
        # Set the duration for the video clip
        clip = clip.set_duration(duration_per_image)
        print('duration set')
        # Apply the zoom-in effect using the 'resize' method
        # Here, `zoom_factor` defines how much the image will zoom in progressively
        clip = clip.resize(lambda t: 1 + zoom_factor * t)  # t increases over time
        clips.append(clip)
        print('clip appended')

    # Concatenate all clips into one video
    final_clip = concatenate_videoclips(clips, method="compose")
    print('final clip concatenated')
    final_clip.fps=24
    
    # Write the video file
    final_clip.write_videofile(output_file) #, fps=24, codec="libx264")
    return final_clip


def generate_zoom_video(image_folder, output_file, zoom_factor=0.05, duration_per_image=1):
    '''Function to generate the zoom video from images in a folder'''
    # Get all image files from the provided folder
    image_files = [filename for filename in os.listdir(image_folder) if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print("No image files found in the provided directory.")
        return

    input_files = [os.path.join(image_folder, filename) for filename in image_files]

    create_zoom_video(input_files, output_file, zoom_factor, duration_per_image)
    print(f"Video saved to {output_file}")


In [ ]:
# UNCOMMENT TO TEST
"""random.seed(4)
prompt_concrete = generate_prompt(PromptType.CONCRETE, 5)
inpainting_filenames = inpainting_pipeline(prompt_concrete)
create_zoom_video(inpainting_filenames[::-1], 'inpainting_testing.mp4', zoom_factor=1.2, duration_per_image=0.8)"""

# Reset memory

In [ ]:
#torch.cuda.empty_cache()
%reset -f